# Creating `h5ad` files
This notebook builds h5ad files for both gene UMI count matrices and guide assignment matrices.

# Set-up

In [1]:
# imports
import os
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.io import mmwrite, mmread
import anndata as ad

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


In [6]:
# paths
path_guide_assignments = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Huangfu_HUES8-embryonic-stemcell-differentiation_TF-Perturb-seq/Perturbation_information/sgrna_design_matrix_filtered_combined_control_final.csv"
path_id_master = "/cellar/users/aklie/data/datasets/tf_perturb_seq/ref/sgRNA_id_master.tsv"

path_out = "/cellar/users/aklie/data/datasets/tf_perturb_seq/datasets/Huangfu_HUES8-embryonic-stemcell-differentiation_TF-Perturb-seq/Perturbation_information"
os.makedirs(path_out, exist_ok=True)

# Load auxiliary data

In [7]:
# Load the id mapping
id_master = pd.read_csv(path_id_master, sep="\t")
id_map = id_master.set_index("Huangfu_id")["protospacer_ID"].to_dict()

In [8]:
id_master["type"].value_counts()

type
targeting           13147
non_targeting         600
negative_control      598
positive_control       19
Name: count, dtype: int64

# Load guide assignments

In [9]:
# Read the guide assignments for this dataset
guide_assignments = pd.read_csv(path_guide_assignments, delim_whitespace=True, engine="python")
guide_ids = guide_assignments["gene_ids"]
guide_assignments.shape

/tmp/ipykernel_1859328/1152691566.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  guide_assignments = pd.read_csv(path_guide_assignments, delim_whitespace=True, engine="python")


(13167, 95718)

In [11]:
# Check how many guides are in the master
in_master_msk = guide_ids.isin(id_master["Huangfu_id"])
in_master_msk.sum()

13166

In [12]:
# which ones are not in the master?
guide_assignments[~in_master_msk]

,gene_ids,AAACCCAAGAACGCGT-1,AAACCCAAGATACTGA-1,AAACCCACAGGCGTTC-1,AAACCCAGTAGGTCAG-1,AAACCCAGTATGTCAC-1,AAACCCAGTCATCGCG-1,AAACCCAGTCGACTGC-1,AAACCCAGTCGTTGGC-1,AAACCCAGTGGCTGCT-1,...,TTTGTTGAGCCATGCC-8,TTTGTTGCACTACTTT-8,TTTGTTGCAGTTGTCA-8,TTTGTTGCATGTAACC-8,TTTGTTGCATTCATCT-8,TTTGTTGGTCTGGTTA-8,TTTGTTGGTGGCTACC-8,TTTGTTGTCACAACCA-8,TTTGTTGTCCGGACGT-8,TTTGTTGTCTCTGGTC-8
13166,control,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [13]:
# 
guide_assignments[~in_master_msk].iloc[0].drop("gene_ids").sum()

8093

In [14]:
# Subset the guide assignments to only include those in the master
guide_assignments_subset = guide_assignments[in_master_msk]
guide_assignments_subset.shape

(13166, 95718)

In [15]:
# Any troublesome NaNs?
guide_assignments_subset.isna().sum().sum()

0

In [16]:
# Convert to sparse matrix
guide_assignments_sparse = csr_matrix(guide_assignments_subset.drop(columns=["gene_ids"]).values)
guide_assignments_sparse.shape

(13166, 95717)

# Write 10X matrix format

In [17]:
# Write features.tsv.gz
features = guide_assignments_subset["gene_ids"]
features = features.map(id_map).values
print(f"Number of features: {len(features)}")
print(f"First 5 features: {features[:5]}")
pd.Index(features).to_frame().to_csv(f"{path_out}/guide_assignment_matrix/features.tsv.gz", sep="\t", header=False, index=False, compression="gzip")

Number of features: 13166
First 5 features: ['CD81_strong' 'CD81_weak' 'CD151_strong' 'CD151_weak' 'CD55']


In [18]:
# Barcodes
barcodes = guide_assignments_subset.drop(columns=["gene_ids"]).columns
print(f"Number of barcodes: {len(barcodes)}")
print(f"First 5 barcodes: {barcodes[:5]}")
barcodes.to_frame().to_csv(f"{path_out}/guide_assignment_matrix/barcodes.tsv.gz", sep="\t", header=False, index=False, compression="gzip")

Number of barcodes: 95717
First 5 barcodes: Index(['AAACCCAAGAACGCGT-1', 'AAACCCAAGATACTGA-1', 'AAACCCACAGGCGTTC-1',
       'AAACCCAGTAGGTCAG-1', 'AAACCCAGTATGTCAC-1'],
      dtype='object')


In [19]:
len(features), len(barcodes)

(13166, 95717)

In [20]:
guide_assignments_sparse.T.shape

(95717, 13166)

In [21]:
# Write mtx.mtx, features.tsv.gz, barcodes.tsv.gz
mmwrite(f"{path_out}/guide_assignment_matrix/matrix.mtx", guide_assignments_sparse.T)

# Make AnnData

In [22]:
# Reload the matrix to check
mtx = mmread(f"{path_out}/guide_assignment_matrix/matrix.mtx").tocsr()
mtx.shape

(95717, 13166)

In [23]:
# Reload the features and barcodes
barcodes = pd.read_csv(
    f"{path_out}/guide_assignment_matrix/barcodes.tsv.gz", sep="\t", header=None, index_col=0
)
barcodes.index.name = "barcode"
features = pd.read_csv(
    f"{path_out}/guide_assignment_matrix/features.tsv.gz", sep="\t", header=None, index_col=0
)
features.index.name = "feature"

In [24]:
# Construct AnnData object
adata = ad.AnnData(
    X=mtx,
    obs=barcodes,
    var=features,
)

/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:183: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/cellar/users/aklie/opt/miniconda3/envs/scverse-lite-py39/lib/python3.9/site-packages/anndata/_core/anndata.py:1908: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [25]:
# add in guide metadata
adata.var = adata.var.merge(id_master, left_index=True, right_on="protospacer_ID", how="left").set_index("protospacer_ID")
adata

AnnData object with n_obs × n_vars = 95717 × 13166
    var: 'protospacer', 'intended_target_name', 'genomic_element', 'reverse_compliment', 'Huangfu_id', 'Huangfu_name', 'type'

In [26]:
# Write AnnData object
adata.write(f"{path_out}/guide_assignment_matrix.h5ad")

In [27]:
# Write csv for PySpade
adata.to_df().T.to_csv(f"{path_out}/guide_assignment_matrix.csv")

# DONE!

---